In [ ]:
import numpy as np

In [ ]:
# Initialize parameters
S0 = 105      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
r = 0.05      # annual risk-free rate
q = 0         # dividend yield
vol = 0.25    # volatility
steps = 10     # number of time steps
cp = "call"   # type of option

In [ ]:
dt = T / steps                 # length of time-step
u = np.exp(vol * np.sqrt(dt))  # up factor
d = 1.0 / u                    # down factor
a = np.exp((r-q)*dt)           # growth factor
p = (a - d) / (u - d)          # risk-neutral up probability
disc = np.exp(-r*dt)           # discount factor

In [ ]:
S = S0 * (d**np.arange(steps, -1, -1)) * (u**np.arange(0, steps+1, 1))
S

array([ 47.62657649,  55.78498896,  65.34093404,  76.53380848,
        89.64401759, 105.        , 122.98645571, 144.05398371,
       168.73037036, 197.63381164, 231.48840021])

In [ ]:
# option payoff at expiration
if cp == 'put':
    V = np.maximum(0, K - S)
else:
    V = np.maximum(0, S - K)
V

array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   5.        ,  22.98645571,  44.05398371,
        68.73037036,  97.63381164, 131.48840021])

In [ ]:
# backward recursion through the tree
for i in np.arange(steps-1,-1,-1):
    S = (S * u)[0:-1]
    V =  disc * (p*V[1:i+2] + (1-p)*V[0:i+1])
    #print(V)
    if cp == 'put':
        V = np.maximum(V, K - S)
    else:
        V = np.maximum(V, S - K)
V

array([15.7978638])

## As Function

In [ ]:
def crr_vanilla(cp, K, T, S0, vol, r, q, steps=1000, american=False):
    dt = T / steps                 # length of time-step
    u = np.exp(vol * np.sqrt(dt))  # up factor
    d = 1.0 / u                    # down factor
    a = np.exp((r-q)*dt)           # growth factor
    p = (a - d) / (u - d)          # risk-neutral up probability
    disc = np.exp(-r*dt)           # discount factor

    # initializing terminal stock values
    S = S0 * (d**np.arange(steps, -1, -1)) * (u**np.arange(0, steps+1, 1))

    # value of option at final time step
    if cp == 'put':
        V = np.maximum(0, K - S)
    else:
        V = np.maximum(0, S - K)


    # backward recursion through the tree
    for i in np.arange(steps-1,-1,-1): 
        S = (S * u)[0:-1] # a tricky way of calculating the previous underlying prices
        V =  disc * (p*V[1:i+2] + (1-p)*V[0:i+1])
        # check for early exercise
        if american:
            if cp == 'put':
                V = np.maximum(V, K - S)
            else:
                V = np.maximum(V, S - K)

    
    return V[0]

In [ ]:
# Initialize parameters
S0 = 100      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
r = 0.1      # annual risk-free rate
q = 0        # dividend yield
vol = 0.2    # volatility
steps = 1000     # number of time steps
cp = "put"   # type of option
american=False
crr_vanilla(cp, K, T, S0, vol, r, q, steps, american)

3.7513304406239656

In [ ]:
from py_vollib.black_scholes_merton import black_scholes_merton
from py_vollib.black_scholes_merton.greeks.analytical import delta

In [ ]:
black_scholes_merton('p', S0, K, T, r, vol, q)

3.753418388256841

In [ ]:
delta('p', S0, K, T, r, vol, q)

-0.2742531177500736

## Adding Delta

In [ ]:
def crr_vanilla(cp, K, T, S0, vol, r, q, american=False, steps=1000):
    dt = T / steps                 # length of time-step
    u = np.exp(vol * np.sqrt(dt))  # up factor
    d = 1.0 / u                    # down factor
    a = np.exp((r-q)*dt)           # growth factor
    p = (a - d) / (u - d)          # risk-neutral up probability
    disc = np.exp(-r*dt)           # discount factor

    # initializing terminal underlying values
    S = S0 * (d**np.arange(steps, -1, -1)) * (u**np.arange(0, steps+1, 1))

    # value of option at final time step
    if cp == 'put':
        V = np.maximum(0, K - S)
    else:
        V = np.maximum(0, S - K)


    # backward recursion through the tree (only to the penultimate step so we can calculate delta)
    for i in np.arange(steps-1,0,-1): 
        S = (S * u)[0:-1] # a tricky way of calculating the previous underlying prices
        V =  disc * (p*V[1:i+2] + (1-p)*V[0:i+1])
        # check for early exercise
        if american:
            if cp == 'put':
                V = np.maximum(V, K - S)
            else:
                V = np.maximum(V, S - K)

    delta = (V[1] - V[0]) / (S[1] - S[0])
    price = disc * (p*V[1] + (1-p)*V[0])
    
    return price, delta

In [ ]:
# Initialize parameters
S0 = 100      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
r = 0.1      # annual risk-free rate
q = 0        # dividend yield
vol = 0.2    # volatility
steps = 10000     # number of time steps
cp = "put"   # type of option
american=False
crr_vanilla(cp, K, T, S0, vol, r, q, american, steps)

(3.753209570128056, -0.274257271897797)

## Implied Volatility

In [ ]:
# Initialize parameters
px = 11.10   # option price
S0 = 100      # initial stock price
K = 100       # strike price
T = 0.5         # time to maturity in years
r = 0.03     # annual risk-free rate
q = 0.01        # dividend yield
#vol = 0.2    # volatility
steps = 1000     # number of time steps
cp = "call"   # type of option
american=True

In [ ]:
from scipy.optimize import fsolve

In [ ]:
def difference(vol):
    opt_px = crr_vanilla(cp, K, T, S0, vol, r, q, american, steps)[0]
    return opt_px - px

In [ ]:
difference(0.19)

-5.289094414747562

In [ ]:
fsolve(difference, 0.1)[0]

0.38069093405733784

In [ ]:
def implied_vol(cp, K, T, S0, px, r, q, american, steps):
    def difference(vol):
        opt_px = crr_vanilla(cp, K, T, S0, vol, r, q, american, steps)[0]
        return opt_px - px
    iv = fsolve(difference, 0.1)[0]
    return iv

In [ ]:
%%timeit
implied_vol(cp, K, T, S0, px, r, q, american, steps)

73.6 ms ± 221 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Testing a Real Life Option

In [ ]:
underlying = 'QQQ'
current_date = '11/16/2018'
expiration = '12/21/2018'
cp = 'put'
K = 160
S0 = 168
d2x = 24
px = 2.25
T = 24/252
r = 0
q = 0
american = False
steps = 10000

In [ ]:
implied_vol(cp, K, T, S0, px, r, q, american, steps)

0.2636373413579189